# AABW Bottom Age Plot

Compare speed of different methods of extracting bottom age

In [25]:
%matplotlib inline

import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import matplotlib.path as mpath
import cosima_cookbook as cc
import cmocean.cm as cm

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [2]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:34493 Dashboard: /proxy/8787/status,Cluster Workers: 8 Cores: 48 Memory: 188.57 GiB


In [13]:
master_session = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')
expt = '01deg_jra55v13_ryf9091'

start_time = '1950-01-01'
end_time='1950-01-31'

lat_slice  = slice(-80,-47)

Now we want to get the age in the bottom cell, normalised by the simulation age.

In [14]:
# import age:
age = cc.querying.getvar(expt,'age_global',master_session,
                     start_time=start_time, end_time=end_time, ncfile = 'ocean.nc')
# more accurately select the right time period and limit to south of 50S:
age = age.sel(time=slice(start_time,end_time)).sel(yt_ocean=lat_slice)
# remove singleton dimension:
age = age.squeeze()

In [25]:
# find max age and normalise so that is equal to 1:
#age_max = age_mean.max()
#age_mean = age_mean / age_max

## Method 1:

In [33]:
%%time

age = cc.querying.getvar(expt,'age_global',master_session,
                     start_time=start_time, end_time=end_time, ncfile = 'ocean.nc')
# more accurately select the right time period and limit to south of 50S:
age = age.sel(time=slice(start_time,end_time)).sel(yt_ocean=lat_slice)
# remove singleton dimension:
age = age.squeeze()

# select out bottom values:
depth_array = age * 0 + age.st_ocean
max_depth = depth_array.max(dim = 'st_ocean', skipna= True)
bottom_age = age.where(depth_array.st_ocean >= max_depth)
bottom_age = bottom_age.sum(dim = 'st_ocean')
bottom_age = bottom_age.load()

CPU times: user 4.53 s, sys: 519 ms, total: 5.05 s
Wall time: 5.57 s


## Method 2:

In [23]:
%%time

age = cc.querying.getvar(expt,'age_global',master_session,
                     start_time=start_time, end_time=end_time, ncfile = 'ocean.nc')
# more accurately select the right time period and limit to south of 50S:
age = age.sel(time=slice(start_time,end_time)).sel(yt_ocean=lat_slice)
# remove singleton dimension:
age = age.squeeze()

# bottom cell indices:
kmt = cc.querying.getvar(expt,'kmt',master_session, n=-1)
kmt = kmt.sel(yt_ocean=lat_slice)
kmt = kmt.fillna(1.0).astype(int) - 1
kmt.load()

bottom_age = age.isel(st_ocean=kmt)
bottom_age = bottom_age.load()

distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

CPU times: user 24.4 s, sys: 1.16 s, total: 25.6 s
Wall time: 27.7 s


## Method 2 with loading age first:

In [24]:
%%time

age = cc.querying.getvar(expt,'age_global',master_session,
                     start_time=start_time, end_time=end_time, ncfile = 'ocean.nc')
# more accurately select the right time period and limit to south of 50S:
age = age.sel(time=slice(start_time,end_time)).sel(yt_ocean=lat_slice)
# remove singleton dimension:
age = age.squeeze()

# bottom cell indices:
kmt = cc.querying.getvar(expt,'kmt',master_session, n=-1)
kmt = kmt.sel(yt_ocean=lat_slice)
kmt = kmt.fillna(1.0).astype(int) - 1
kmt.load()

age = age.load()
bottom_age = age.isel(st_ocean=kmt)
bottom_age = bottom_age.load()

CPU times: user 8.21 s, sys: 1.97 s, total: 10.2 s
Wall time: 10.6 s


distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)


## Method 3:

In [31]:
%%time
age = cc.querying.getvar(expt,'age_global',master_session,
                     start_time=start_time, end_time=end_time, ncfile = 'ocean.nc')
# more accurately select the right time period and limit to south of 50S:
age = age.sel(time=slice(start_time,end_time)).sel(yt_ocean=lat_slice)
# remove singleton dimension:
age = age.squeeze()

bottom_mask = age.where(~xr.ufuncs.isfinite(age.shift({'st_ocean':-1})))
bottom_mask = ~xr.ufuncs.isnan(bottom_mask).to_dataset(name='umask')
bottom_age = age.where(bottom_mask.umask).sum('st_ocean')
bottom_age = bottom_age.load()

distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)


CPU times: user 4.67 s, sys: 506 ms, total: 5.18 s
Wall time: 5.52 s
